In [1]:
import gdown

In [2]:
url='https://drive.google.com/file/d/1Y6dBQTBGdl5by0IHLMUrPlHW-ytPdT2B/view?usp=sharing'

In [3]:
file_id = url.split('/')[5]

In [4]:
prefix_url = f'https://drive.google.com/uc?/export=download$id={file_id}'

In [5]:
gdown.download(prefix_url,"Kidney_Image_Classification_Data.zip")

Downloading...
From: https://drive.google.com/uc?/export=download$id=1Y6dBQTBGdl5by0IHLMUrPlHW-ytPdT2B
To: /Users/krushna/Kidney-Disease-Classification/research/Kidney_Image_Classification_Data.zip
1.70kB [00:00, 1.35MB/s]


'Kidney_Image_Classification_Data.zip'

In [6]:
import os  

In [7]:
%pwd

'/Users/krushna/Kidney-Disease-Classification/research'

In [8]:
os.chdir("../")

In [9]:
%pwd

'/Users/krushna/Kidney-Disease-Classification'

In [10]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [11]:
from Kidney_Image_Classifier.constants import *
from Kidney_Image_Classifier.utils.overlapping import create_directories,read_yaml

In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [13]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from Kidney_Image_Classifier.utils.overlapping import get_size

In [14]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-10-05 02:04:55,608: INFO: overlapping: yaml file: config/config.yaml loaded successfully]
[2024-10-05 02:04:55,612: INFO: overlapping: yaml file: params.yaml loaded successfully]
[2024-10-05 02:04:55,613: INFO: overlapping: created directory at: artifacts]
[2024-10-05 02:04:55,614: INFO: overlapping: created directory at: artifacts/data_ingestion]
[2024-10-05 02:04:55,615: INFO: 986331348: Downloading data from https://drive.google.com/file/d/1Y6dBQTBGdl5by0IHLMUrPlHW-ytPdT2B/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1Y6dBQTBGdl5by0IHLMUrPlHW-ytPdT2B
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1Y6dBQTBGdl5by0IHLMUrPlHW-ytPdT2B&confirm=t&uuid=2c6a3d49-a2f6-4979-b2a3-aa53d149d194
To: /Users/krushna/Kidney-Disease-Classification/artifacts/data_ingestion/data.zip
100%|██████████████████████████████████████████████████████| 944M/944M [00:41<00:00, 22.9MB/s]

[2024-10-05 02:05:40,990: INFO: 986331348: Downloaded data from https://drive.google.com/file/d/1Y6dBQTBGdl5by0IHLMUrPlHW-ytPdT2B/view?usp=sharing into file artifacts/data_ingestion/data.zip]
